In [5]:
import torch
import sklearn
import numpy as np
import gpytorch
from gpytorch.models import ExactGP
from torch.distributions import Normal, MultivariateNormal
from matplotlib import pyplot as plt
from skgpytorch.models import ExactGPRegressor
from gpytorch.kernels import ScaleKernel, RBFKernel
from gpytorch.constraints import GreaterThan
from sklearn.neighbors import NearestNeighbors
import pandas as pd
device = 'cuda'

In [ ]:
N = 1024
torch.manual_seed(0)
x_dist = Normal(torch.tensor([0.0]), torch.tensor([5.0]))
X = x_dist.sample((N,))
K = ScaleKernel(RBFKernel())
K.base_kernel.lengthscale = 0.5
K.outputscale = 4.0
K_ = K(X,X) + (1**2)*torch.eye(len(X))
dist = MultivariateNormal(torch.zeros((N,)),K_.evaluate())
Y = dist.sample()
plt.scatter(X,Y)

In [ ]:
class SGDGP(ExactGPRegressor):
    def __init__(self, train_x, train_y, mll):
        super().__init__(train_x, train_y, mll)
    
    def sgd_fit(self, batch_size, lr, n_epochs, n_restarts, thetas, random_state):

        torch.manual_seed(random_state)
        ## creating nn_indices
        neigh = NearestNeighbors(n_neighbors=batch_size, algorithm='kd_tree')
        neigh.fit(self.train_x.cpu())
        _, neigh_idx = neigh.kneighbors(self.train_x.cpu(), batch_size)

        if (len(self.train_x) % batch_size  == 0):
          num_batches = int(len(self.train_x)/batch_size)
        else:
          num_batches = int((len(self.train_x)/batch_size)) + 1

        least_loss = float("inf")
        best_mll_state = None
        self.history["total_loss"] = []
        self.history["signal_var"] = []
        self.history["noise_var"] = []
        self.mll.train()
        for restart in range(n_restarts):
            self.optimizer = torch.optim.SGD(self.mll.parameters(), lr=lr)
            # self.optimizer = torch.optim.Adam(self.mll.parameters(), lr=0.1)
            self.history["total_loss"].append([])
            self.history["signal_var"].append([])
            self.history["noise_var"].append([])

            # Resetting the model for restarts
            if restart > 0:  
                for param in self.mll.parameters():
                    torch.nn.init.normal_(param, mean=0.0, std=1.0)
            self.mll.model.initialize(**thetas)
            self.mll.model.covar_module.base_kernel.raw_lengthscale.requires_grad = False
            for epoch in range(1, n_epochs+1):
                loss = 0
                for iteration in range(1, num_batches+1):
           
                    idx = torch.tensor((iteration - 1)%self.train_x.shape[0])
                    indices = neigh_idx[idx,]
                    X_batch = self.train_x[indices,]
                    y_batch = self.train_y[indices,]

                    self.optimizer.zero_grad()
                    batch_loss = self.loss_func(X_batch, y_batch)
                    self.history["signal_var"][restart].append(self.mll.model.covar_module.outputscale.item())
                    self.history["noise_var"][restart].append(self.mll.likelihood.noise_covar.noise.item())

                    batch_loss.backward()
                    loss += batch_loss.item()

                    self.mll.model.covar_module.raw_outputscale.grad*= batch_size/(3*torch.log(torch.tensor(batch_size)))

                    self.optimizer.step()
                    for group in self.optimizer.param_groups:
                        group['lr'] = lr/(iteration)
                
                loss = loss / num_batches
                self.history["total_loss"][restart].append(loss)

            # Check if best loss
            if loss < least_loss:
                self.best_restart = restart
                least_loss = loss
                best_mll_state = self.mll.state_dict()

        # Load the best model
        if best_mll_state is not None:
            self.mll.load_state_dict(best_mll_state)


In [ ]:
N = 1024
batch_size =128
theta1 = {'likelihood.noise_covar.noise': torch.tensor(3).to(device),
        'covar_module.base_kernel.lengthscale': torch.tensor(0.5).to(device),
        'covar_module.outputscale': torch.tensor(5).to(device),}
theta2 = {'likelihood.noise_covar.noise': torch.tensor(3.5).to(device),
        'covar_module.base_kernel.lengthscale': torch.tensor(0.5).to(device),
        'covar_module.outputscale': torch.tensor(2.5).to(device),}
theta3 = {'likelihood.noise_covar.noise': torch.tensor(2.7).to(device),
        'covar_module.base_kernel.lengthscale': torch.tensor(0.5).to(device),
        'covar_module.outputscale': torch.tensor(2.5).to(device),}

thetas = [theta1,theta2,theta3]
lrs = [3,3,1]
fig,ax = plt.subplots(1,3,figsize=(30,6))

for j in range(len(thetas)):
    theta = thetas[j]
    for i in range(10):
        lr = lrs[j]
        torch.manual_seed(0)
        x_dist = Normal(torch.tensor([0.0]), torch.tensor([5.0]))
        X = x_dist.sample((N,))
        K = ScaleKernel(RBFKernel())
        K.base_kernel.lengthscale = 0.5
        K.outputscale = 1.0
        cov = K(X,X) + (0.5)*(torch.eye(len(X)))
        dist = MultivariateNormal(torch.zeros((1024)),cov.evaluate())
        torch.manual_seed(i+1)
        Y = dist.sample()
        
        kernel = ScaleKernel(RBFKernel(ard_num_dims=X.shape[1])).to(device)
        model = SGDGP(X.to(device), Y.to(device), kernel).to(device)
        model.sgd_fit(batch_size,lr,50,1,thetas = theta, random_state=0)
        ax[j].plot(pd.Series(model.history['signal_var'][0]),linestyle='dashed',linewidth=1.5) 
        ax[j].plot(pd.Series(model.history['noise_var'][0]),linestyle='dotted',linewidth=1.5)
    ax[j].axhline(y = 1.0, color = 'black', linestyle = 'dashed')
    ax[j].axhline(y = 0.5, color = 'red', linestyle = ':')
    ax[j].axhline(y = 1.5, color = 'black', linestyle = 'dashed')
    ax[j].set_xlabel('Iteration (k)')
    ax[j].set_ylabel('O(k)')
    ax[j].set_title(f'({theta["covar_module.outputscale"]},{round(theta["likelihood.noise_covar.noise"].item(),2)})')
    # ax[j].set_xticks([0,50,100,150,200])
    ax[j].set_yticks([0,1,2,3,4,5])
    ax[j].grid()

plt.savefig('figure1_testing.png')